<a href="https://colab.research.google.com/github/hardlyrichie/data-science-gss/blob/master/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring factors impacting mental health 

Data Science Project 2

Richard Gao

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Exploring the Data

I setup the `read_stata_dict` function provided by Allen Downey to read in GSS strata data into Pandas. 

In [0]:
import re

def read_stata_dict(fp, **options):
    """Reads a Stata dictionary file.

    fp: open file pointer
    options: dict of options passed to open()

    returns: DataFrame
    """
    type_map = dict(byte=int, int=int, long=int, float=float,
                    double=float, numeric=float)

    var_info = []
    for line in fp:
        match = re.search(r'_column\(([^)]*)\)', line)
        if not match:
            continue
        start = int(match.group(1))
        t = line.split()
        vtype, name, fstring = t[1:4]
        name = name.lower()
        if vtype.startswith('str'):
            vtype = str
        else:
            vtype = type_map[vtype]
        long_desc = ' '.join(t[4:]).strip('"')
        var_info.append((start, vtype, name, fstring, long_desc))

    columns = ['start', 'type', 'name', 'fstring', 'desc']
    variables = pd.DataFrame(var_info, columns=columns)

    # fill in the end column by shifting the start column
    # NOTE: the last column doesn't work
    variables['end'] = variables.start.shift(-1, fill_value=0)

    return variables